In [4]:
import pinecone
import os

# Initialize Pinecone
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Create a Pinecone client instance
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

In [5]:
index_name = "emergency-index"
pc.create_index(
    name=index_name,
    dimension=1024,  # Dimension for the multilingual-e5-large model
    metric="cosine",
    spec=pinecone.ServerlessSpec(
        cloud="aws",
        region="us-east-1"  # Choose the region according to your requirements
    )
)

In [6]:
import json
with open('emergencies.json') as json_file:
    data = json.load(json_file)


In [16]:
# Prepare the data for upsert
upsert_data = []
for emergency in data['emergencies']:
    emergency_text = emergency['emergency']
    next_step = emergency['next_step']
    
    # Embed using Pinecone's built-in embedding with the required input_type parameter
    embedding_response = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[emergency_text],
        parameters={"input_type": "passage"}  # Specify the input type
    )
    
    # Access the embedding directly from the response
    vector = embedding_response[0]['values']  # Get the first embedding's values

    # Append the data in the required format for upsert
    upsert_data.append({
        "id": emergency_text,  # Using the emergency text as the ID
        "values": vector,  # Use the embedding values directly
        "metadata": {"next_step": next_step}  # Metadata for the embedding
    })

# Upsert data into Pinecone
index = pc.Index(index_name)
index.upsert(vectors=upsert_data)




{'upserted_count': 52}

In [26]:
def query_emergency(emergency_query):
    # Embed the query text using Pinecone's built-in embedding with the required input_type parameter
    query_vector = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[emergency_query],
        parameters={"input_type": "passage"}  # Specify the input type
    )[0]['values']  # Get the embedding

    # Query the Pinecone index
    result = index.query(
        vector=query_vector,
        top_k=5,
        include_metadata=True
    )
    return result

# Example of querying the database
emergency_query = "Not getting"
response = query_emergency(emergency_query)

# Print the next step
if response['matches']:
    next_step = response['matches'][0]['metadata']['next_step']
    print(f"Next step: {next_step}")
else:
    print("No matching emergency found.")


Next step: Help the patient sit or lie down in a comfortable position. Offer small sips of clear fluids, like water or ginger ale. Seek medical help if nausea persists or worsens.
